In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import torch

/opt/conda/envs/dkt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
from sklearn.preprocessing import LabelEncoder
def categorical_feature(data : pd.DataFrame,columns : list) -> pd.DataFrame:
    for col in columns:
        X = data[col]
        enc = LabelEncoder()
        enc.fit(X)
        data[col] = enc.transform(X)
    return data


In [2]:
train_df = pd.read_pickle('/opt/ml/input/data/after_fe_train_test.pkl')
train_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,month,...,test_mean,test_std,test_sum,tag_mean,tag_std,tag_sum,kind,solvesec,time_category,solvesec_3600
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,NaN,0,NaN,3,...,0.947683,0.222749,1268,0.955022,0.207410,637,train,3.0,1,3.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,1,1.000000,3,...,0.947683,0.222749,1268,0.913187,0.281603,3040,train,8.0,3,8.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,2.0,2,1.000000,3,...,0.947683,0.222749,1268,0.913187,0.281603,3040,train,7.0,2,7.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,3.0,3,1.000000,3,...,0.947683,0.222749,1268,0.913187,0.281603,3040,train,7.0,2,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,4.0,4,1.000000,3,...,0.947683,0.222749,1268,0.913187,0.281603,3040,train,11.0,4,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1.0,4,0.250000,6,...,0.662590,0.472996,921,0.689706,0.462671,2814,train,6632178.0,9,3600.0
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1.0,5,0.200000,8,...,0.655109,0.475550,718,0.697874,0.459253,2199,train,11.0,4,11.0
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2.0,6,0.333333,8,...,0.655109,0.475550,718,0.697874,0.459253,2199,train,46.0,4,46.0
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3.0,7,0.428571,8,...,0.655109,0.475550,718,0.697874,0.459253,2199,train,73.0,5,73.0


In [3]:
train_df.loc[train_df['answerCode'] == -1]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,month,...,test_mean,test_std,test_sum,tag_mean,tag_std,tag_sum,kind,solvesec,time_category,solvesec_3600


In [23]:
train_df = train_df[['userID', 'assessmentItemID', 'testId', 'answerCode', 
       'KnowledgeTag', 'user_correct_answer', 'user_total_answer', 'user_acc',
       'month', 'day', 'hour', 'big_category', 'problem_num',
       'mid_category', 'test_mean', 'test_std', 'test_sum', 'tag_mean',
       'tag_std', 'tag_sum', 'time_category', 'solvesec_3600', 'kind']]

In [24]:
train_df.to_csv('/opt/ml/input/data/FE_train_test.csv')

In [26]:
assessmentItemID_to_idx = {v:k for k,v in enumerate(train_df['assessmentItemID'].unique())}
idx_to_assessmentItemID = {k:v for k,v in enumerate(train_df['assessmentItemID'].unique())}

testId_to_idx = {v:k for k,v in enumerate(train_df['testId'].unique())}
idx_to_testId = {k:v for k,v in enumerate(train_df['testId'].unique())}

train_df['assessmentItemID'] = train_df['assessmentItemID'].map(assessmentItemID_to_idx)
train_df['testId'] = train_df['testId'].map(testId_to_idx)

# train_df['user_correct_answer'].fillna(0,inplace=True)
# train_df['user_acc'].fillna(method='bfill',inplace= True)
train_df = train_df.loc[train_df['answerCode'] != -1]
train_df

/tmp/ipykernel_78304/682230317.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['assessmentItemID'] = train_df['assessmentItemID'].map(assessmentItemID_to_idx)
/tmp/ipykernel_78304/682230317.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['testId'] = train_df['testId'].map(testId_to_idx)


,userID,assessmentItemID,testId,answerCode,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,month,day,...,problem_num,mid_category,test_mean,test_std,test_sum,tag_mean,tag_std,tag_sum,time_category,solvesec_3600
0,0,0,0,1,7224,NaN,0,NaN,3,24,...,1,1,0.947683,0.222749,1268,0.955022,0.207410,637,1,3.0
1,0,1,0,1,7225,1.0,1,1.000000,3,24,...,2,1,0.947683,0.222749,1268,0.913187,0.281603,3040,3,8.0
2,0,2,0,1,7225,2.0,2,1.000000,3,24,...,3,1,0.947683,0.222749,1268,0.913187,0.281603,3040,2,7.0
3,0,3,0,1,7225,3.0,3,1.000000,3,24,...,4,1,0.947683,0.222749,1268,0.913187,0.281603,3040,2,7.0
4,0,4,0,1,7225,4.0,4,1.000000,3,24,...,5,1,0.947683,0.222749,1268,0.913187,0.281603,3040,4,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,4499,713,0,438,1.0,4,0.250000,6,5,...,5,71,0.662590,0.472996,921,0.689706,0.462671,2814,9,3600.0
2526696,7441,1286,199,1,8836,1.0,5,0.200000,8,21,...,1,165,0.655109,0.475550,718,0.697874,0.459253,2199,4,11.0
2526697,7441,1287,199,1,8836,2.0,6,0.333333,8,21,...,2,165,0.655109,0.475550,718,0.697874,0.459253,2199,4,46.0
2526698,7441,1288,199,1,8836,3.0,7,0.428571,8,21,...,3,165,0.655109,0.475550,718,0.697874,0.459253,2199,5,73.0


In [27]:
cat_features_ = ['userID', 'assessmentItemID', 'testId', 'answerCode', 
       'KnowledgeTag', 'month', 'day', 'hour', 'big_category', 'problem_num',
       'mid_category', 'time_category']
# cat_features = ['userID', 'assessmentItemID', 'testId', 'answerCode', 
#        'KnowledgeTag', 'user_correct_answer', 'user_total_answer', 'user_acc',
#        'month', 'day', 'hour', 'big_category', 'problem_num',
#        'mid_category', 'test_mean', 'test_std', 'test_sum', 'tag_mean',
#        'tag_std', 'tag_sum', 'time_category', 'solvesec_3600']
train_df = categorical_feature(train_df,cat_features_)

In [28]:
# Ratio is 6/24 = 25%
valid_df = train_df.groupby('userID').tail(6)
train_df.drop(valid_df.index, inplace = True)

In [29]:
print(valid_df.shape)
print(train_df.shape)

(44652, 22)
(2481304, 22)


In [30]:
features = ['userID','assessmentItemID', 'testId', 'answerCode', 
       'KnowledgeTag', 'user_correct_answer', 'user_total_answer', 'user_acc',
       'month', 'day', 'hour', 'big_category', 'problem_num',
       'mid_category', 'test_mean', 'test_std', 'test_sum', 'tag_mean',
       'tag_std', 'tag_sum', 'time_category', 'solvesec_3600']

params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'task_type': 'GPU' if torch.cuda.is_available() else 'CPU',
    'grow_policy': 'Lossguide',
    'iterations': 2500,
    'learning_rate': 0.01,
    'random_seed': 42,
    'l2_leaf_reg': 1e-1,
    'depth': 15,
    'max_leaves': 10,
    'border_count': 128,
    'verbose': 50,
}

In [31]:
from catboost import CatBoostClassifier, Pool

# Training and validating data
train_set = Pool(train_df[features],  label = train_df['answerCode'])
val_set = Pool(valid_df[features], label = valid_df['answerCode'])

In [32]:
# Model definition
model = CatBoostClassifier(**params)

# Fitting
model.fit(train_set, eval_set = val_set, use_best_model = True)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 1.0000000	best: 1.0000000 (0)	total: 14.9ms	remaining: 37.1s
50:	test: 1.0000000	best: 1.0000000 (0)	total: 586ms	remaining: 28.2s
100:	test: 1.0000000	best: 1.0000000 (0)	total: 1.12s	remaining: 26.6s
150:	test: 1.0000000	best: 1.0000000 (0)	total: 1.66s	remaining: 25.8s
200:	test: 1.0000000	best: 1.0000000 (0)	total: 2.2s	remaining: 25.2s
250:	test: 1.0000000	best: 1.0000000 (0)	total: 2.75s	remaining: 24.7s
300:	test: 1.0000000	best: 1.0000000 (0)	total: 3.33s	remaining: 24.3s
350:	test: 1.0000000	best: 1.0000000 (0)	total: 3.9s	remaining: 23.9s
400:	test: 1.0000000	best: 1.0000000 (0)	total: 4.49s	remaining: 23.5s
450:	test: 1.0000000	best: 1.0000000 (0)	total: 5.05s	remaining: 23s
500:	test: 1.0000000	best: 1.0000000 (0)	total: 5.64s	remaining: 22.5s
550:	test: 1.0000000	best: 1.0000000 (0)	total: 6.23s	remaining: 22.1s
600:	test: 1.0000000	best: 1.0000000 (0)	total: 6.75s	remaining: 21.3s
650:	test: 1.0000000	best: 1.0000000 (0)	total: 7.25s	remaining: 20.6s
700:	test: 1

In [6]:
df = pd.read_pickle('/opt/ml/input/data/ref_FE_1.pkl')
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,kind,uidIdx,assIdx,testIdx,...,time_category,solvesec_cumsum,solvecumsum_category,big_category_acc,big_category_std,big_category_cumconut,big_category_user_acc,big_category_user_std,big_category_answer,big_category_answer_log1p
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,train,0,5354,975,...,0,0.0,0,0.711898,0.453371,0,0.791908,0.406531,NaN,NaN
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,train,0,5355,975,...,1,3.0,1,0.711898,0.453371,1,0.791908,0.406531,1.0,0.693147
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,train,0,5356,975,...,3,11.0,4,0.711898,0.453371,2,0.791908,0.406531,2.0,1.098612
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,train,0,5357,975,...,2,18.0,4,0.711898,0.453371,3,0.791908,0.406531,3.0,1.386294
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,train,0,5358,975,...,2,25.0,4,0.711898,0.453371,4,0.791908,0.406531,4.0,1.609438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,train,7441,2373,456,...,4,220.0,5,0.701733,0.458110,4,0.200000,0.447214,1.0,0.693147
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,train,7441,3909,748,...,9,3600.0,8,0.679714,0.467279,0,1.000000,0.000000,NaN,NaN
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,train,7441,3910,748,...,4,10.0,3,0.679714,0.467279,1,1.000000,0.000000,1.0,0.693147
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,train,7441,3911,748,...,4,56.0,4,0.679714,0.467279,2,1.000000,0.000000,2.0,1.098612


In [11]:
df.columns # 40개

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'kind', 'uidIdx', 'assIdx', 'testIdx',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'month', 'day',
       'hour', 'dayname', 'big_category', 'problem_num', 'mid_category',
       'test_mean', 'test_std', 'test_sum', 'tag_mean', 'tag_std', 'tag_sum',
       'Timestamp2', 'solvetime', 'solvesec', 'solvesec_3600', 'time_category',
       'solvesec_cumsum', 'solvecumsum_category', 'big_category_acc',
       'big_category_std', 'big_category_cumconut', 'big_category_user_acc',
       'big_category_user_std', 'big_category_answer',
       'big_category_answer_log1p'],
      dtype='object')

In [9]:
FEATS = ['uidIdx',
         'assIdx',
         'testIdx',
         'KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category',
         'solvesec_cumsum',
         'solvecumsum_category',
         'big_category_acc',
         'big_category_std',
         'big_category_cumconut'
        ]

cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
             'problem_num','dayname','month','time_category','solvecumsum_category']
len(FEATS)

27

In [8]:
df.loc[df['answerCode'] == -1]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,kind,uidIdx,assIdx,testIdx,...,time_category,solvesec_cumsum,solvecumsum_category,big_category_acc,big_category_std,big_category_cumconut,big_category_user_acc,big_category_user_std,big_category_answer,big_category_answer_log1p
2815,3,A050133008,A050000133,-1,2020-10-26 13:13:57,5289,test,3,4965,914,...,4,315.0,5,0.658649,0.474735,861,0.653132,0.478682,564.0,6.336826
3531,4,A070146008,A070000146,-1,2020-12-27 02:47:54,9080,test,4,7748,1306,...,4,171.0,5,0.521167,0.500333,425,0.699531,0.464099,299.0,5.703782
10860,13,A070111008,A070000111,-1,2020-12-27 04:35:09,9660,test,13,7484,1271,...,3,103.0,5,0.521167,0.500333,489,0.387755,0.491911,191.0,5.257495
14960,17,A090064006,A090000064,-1,2020-10-30 05:48:37,2611,test,17,9381,1526,...,5,379.0,5,0.454470,0.498214,412,0.920097,0.280270,381.0,5.945421
23479,26,A060135007,A060000135,-1,2020-10-23 11:44:18,1422,test,26,6231,1109,...,4,274.0,5,0.711898,0.453371,334,0.811940,0.398922,273.0,5.613128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525938,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615,test,7395,3692,705,...,1,11.0,4,0.679714,0.467279,9,0.000000,0.471405,1.0,0.693147
2526081,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636,test,7404,2590,496,...,5,218.0,5,0.701733,0.458110,4,0.200000,0.836660,2.0,1.098612
2526282,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402,test,7416,5353,974,...,4,46.0,4,0.658649,0.474735,3,0.250000,0.957427,2.0,1.098612
2526297,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402,test,7417,5353,974,...,4,62.0,5,0.658649,0.474735,3,0.250000,0.957427,2.0,1.098612
